In [ ]:
%pip install azureml-mlflow
%pip install azureml-core
%pip install azure-ai-ml


In [ ]:
import mlflow
import mlflow.azureml
import azureml.mlflow
import azureml.core
from azureml.core import Workspace
from azureml.mlflow import get_portal_url
from mlflow.deployments import get_deploy_client
from azure.identity import DefaultAzureCredential
import os
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
import datetime
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment

In [ ]:
model_name = 'test_model'
model_uri = "runs:/d53ec57a17**/pyfunc_packaged_model"
try:
    os.mkdir(model_name)
except:
    None


model_path = mlflow.tracking.artifact_utils._download_artifact_from_uri(artifact_uri=model_uri, output_path=model_name)

#dbfs:/databricks/mlflow-tracking/2181675471542338/d53ec57a17f2482ab298bbd3ee95b484/artifacts/pyfunc_packaged_model/MLmodel


In [ ]:
workspace_name = "dbxamlws"
resource_group = "databricks-sandbox-rg"
 
subscription_id = "2a**"
tenant_id = "16b3c013-d3**"

DBX_SP_Client_Secret = dbutils.secrets.get(scope="DBX_SP_Credentials",key="DBX_SP_Client_Secret")
DBX_SP_ClientID = dbutils.secrets.get(scope="DBX_SP_Credentials",key="DBX_SP_ClientID")
DBX_SP_TenantID = dbutils.secrets.get(scope="DBX_SP_Credentials",key="DBX_SP_TenantID")

print(f"Test: {DBX_SP_ClientID}")
print(f"Test: {DBX_SP_Client_Secret}")
print(DBX_SP_TenantID)

os.environ["AZURE_CLIENT_ID"] = DBX_SP_ClientID
os.environ["AZURE_CLIENT_SECRET"] = DBX_SP_Client_Secret
os.environ["AZURE_TENANT_ID"] = DBX_SP_TenantID



In [ ]:
print(model_path)


mlflow_model = Model(
    path=model_path,
    type=AssetTypes.MLFLOW_MODEL,
    name=model_name,
    description="Model deployed with V2"
)

In [ ]:
#azml_client = MLClient(
#    DefaultAzureCredential(), subscription_id, resource_group, workspace_name
#)

In [ ]:
#azml_client.models.create_or_update(mlflow_model)

In [ ]:
from azureml.core.authentication import ServicePrincipalAuthentication

svc_pr = ServicePrincipalAuthentication(
                        tenant_id="16b3c013***",
                        service_principal_id= "841ba6d**",
                        service_principal_password="IQG8Q~**")

ws = Workspace(
         subscription_id=subscription_id,
         resource_group=resource_group,
         workspace_name=workspace_name,
         auth=svc_pr
         )

print(ws)

In [ ]:
aml_uri = ws.get_mlflow_tracking_uri()
print(aml_uri)

In [ ]:
import mlflow
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri()) 
print("MLflow tracking URI to point to your Azure ML Workspace setup complete.")



In [ ]:
experiment_name = 'exp-adb-aml-connection2'
mlflow.set_experiment(experiment_name)
print("Experiment setup complete.")

In [ ]:
print("Training model...")
output_folder = 'outputs'
model_file_name = 'california-housing.pkl'
dbutils.fs.mkdirs(output_folder)
model_file_path = os.path.join('/dbfs', output_folder, model_file_name)

In [ ]:
%pip install sklearn-pandas==2.1.0

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing
import os
import numpy as np
import pandas as pd
import pickle
import sklearn
import joblib
import math
import matplotlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

california_housing = fetch_california_housing()
pd_df_california_housing = pd.DataFrame(california_housing.data, columns = california_housing.feature_names) 
pd_df_california_housing['target'] = pd.Series(california_housing.target)

In [ ]:
import mlflow 

mlflow.start_run()  
df = pd_df_california_housing.dropna()  #Drop Missing Values
x_df = df.drop(['target'], axis=1)      #Create Feature Set 
y_df = df['target']                     #Create Label

X_train, X_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=0) #Train-Test Split

numerical = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup','Latitude','Longitude']    #List all Numerical Features

numeric_transformations = [([f], Pipeline(steps=[
  ('imputer', SimpleImputer(strategy='median')),                                         #Imputation and Scaling for all numerical variables.
  ('scaler', StandardScaler())])) for f in numerical]                            

transformations = numeric_transformations

clf = Pipeline(steps=[('preprocessor', DataFrameMapper(transformations, df_out=True)),    #Define the pipeline and its steps
                     ('regressor', GradientBoostingRegressor())])

clf.fit(X_train, y_train)                         #Fit the Model

y_predict = clf.predict(X_test)                   #Predict on the test data
y_actual = y_test.values.flatten().tolist()    

rmse = math.sqrt(mean_squared_error(y_actual, y_predict))   #Calculate and log RMSE
mlflow.log_metric('rmse', rmse)
mae = mean_absolute_error(y_actual, y_predict)    #Calculate and log MAE
mlflow.log_metric('mae', mae)
r2 = r2_score(y_actual, y_predict)                #Calculate and log R2 Score
mlflow.log_metric('R2 score', r2)


mlflow.end_run()  

   #results_graph = os.path.join('/dbfs', output_folder, 'results.png') #Log the graph plotted above
   #plt.savefig(results_graph)
   #mlflow.log_artifact(results_graph)

   #joblib.dump(clf, open(model_file_path,'wb'))      #Log the Model
   #mlflow.log_artifact(model_file_path)

In [ ]:
aml_run = list(ws.experiments[experiment_name].get_runs())[0]


In [ ]:
print(aml_run)